In [151]:
#Library
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()

from pyspark.sql.functions import col, expr
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean
from pyspark.sql.functions import lit
from pyspark.sql.functions import when
from pyspark.sql.functions import asc, desc
#for question 2
from pydataset import data

from pyspark.sql.functions import regexp_extract, regexp_replace
#for question 4
from vega_datasets import data
from pyspark.sql.functions import month, year, quarter

In [14]:
#1. Create a spark data frame that contains your favorite programming languages.
    #- The name of the column should be language
    #- View the schema of the dataframe
    #- Output the shape of the dataframe
    #- Show the first 5 records in the dataframe
import pandas as pd
import numpy as np

data = {'langauge':['python', 'spark', 'sql'], 'favorite':[1, 2, 3]}
df = pd.DataFrame(data)
df.head()
languages = spark.createDataFrame(df)
languages.show()

+--------+--------+
|langauge|favorite|
+--------+--------+
|  python|       1|
|   spark|       2|
|     sql|       3|
+--------+--------+



In [17]:
#2. Load the mpg dataset as a spark dataframe.
mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [45]:
#2.a. Create 1 column of output that contains a message like the one below, for each vehicle:
    # - The 1999 audi a4 has a 4 cylinder engine.
#year, manufacturer, model, cyl,

mpg.select(concat(lit('The '), mpg.year, lit(' '), mpg.manufacturer,
                 lit(' '), mpg.model, lit(' has a '), mpg.cyl, 
                 lit(' cylinder engine.')).alias('vehicle description')
          ).show(5, truncate = False)
    

+-----------------------------------------+
|vehicle description                      |
+-----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 6 cylinder engine.|
+-----------------------------------------+
only showing top 5 rows



In [51]:
#2.b. Transform the trans column so that it only contains either manual or auto.
mpg.filter(mpg.trans.contains('auto')).show()

+------------+------------------+-----+----+---+--------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|   trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+--------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|auto(l5)|  4| 16| 25|  p|compact|
|        audi|        a4 quattro|  2.0|2008|  4|auto(s6)|  4| 19| 27|  p|compact|
|        audi|        a4 quattro|  2.8|1999|  6|auto(l5)|  4| 15| 25|  p|compact|
|        audi|        a4 quattro|  3.1|2008|  6|auto(s6)|  4| 17| 25|  p|compact|
|        audi|        a6 quattro|  2.8|1999|  6|auto(l5)|  4| 15| 24|  p|midsize|
|        audi|  

In [52]:
#3. Load the tips dataset as a spark dataframe.
tips = spark.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [122]:
#3.a. What percentage of observations are smokers?
#no/total smokers
tips.filter(tips.smoker == 'No').count()/tips.count()*100

61.885245901639344

In [142]:
#3.b. Create a column that contains the tip percentage
tip_percentage = tips.select(((tips.tip/tips.total_bill)*100).alias('tip_percentage')).show()

+------------------+
|    tip_percentage|
+------------------+
|5.9446733372572105|
|16.054158607350097|
|16.658733936220845|
| 13.97804054054054|
|14.680764538430255|
| 18.62396204033215|
| 22.80501710376283|
|11.607142857142858|
|13.031914893617023|
|21.853856562922868|
| 16.65043816942551|
|14.180374361883155|
|10.181582360570687|
|16.277807921866522|
|20.364126770060686|
|18.164967562557923|
| 16.16650532429816|
|22.774708410067525|
|20.624631703005306|
|16.222760290556902|
+------------------+
only showing top 20 rows



In [147]:
#3.c. Calculate the average tip percentage for each combination of sex and smoker.
tips.groupBy('sex').agg(mean((tips.tip/tips.total_bill)*100).alias('avg_tip_by_gender')).show()

+------+------------------+
|   sex| avg_tip_by_gender|
+------+------------------+
|Female|16.649073632892478|
|  Male|15.765054700429744|
+------+------------------+



In [150]:
#4. Use the seattle weather dataset referenced in the lesson to answer the questions below.
weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



In [102]:
#Convert the temperatures to fahrenheit.
weather = weather.select(weather.temp_max)

In [103]:
#Which month has the most rain, on average?

In [104]:
#Which year was the windiest?

In [105]:
#What is the most frequent type of weather in January?

In [106]:
#What is the average high and low temperature on sunny days in July in 2013 and 2014?

In [107]:
#What percentage of days were rainy in q3 of 2015?

In [ ]:
#For each year, find what percentage of days it rained (had non-zero precipitation).